## This is new section, where we try to define queries in Natural Language.

Aperturedb has a property graph interface in llama_index repo. will try to use that to query the KG.

In [ ]:
%pip install llama-index-llms-openai
%pip install --upgrade git+https://github.com/aperture-data/llama_index@kg_fixes#subdirectory=llama-index-integrations/graph_stores/llama-index-graph-stores-ApertureDB

In [ ]:
import os
os.environ["APERTUREDB_KEY"] = "WzEsMSwicmFnLXRlc3Qtcmg4ZWNlZmwuZmFybTAwMDQuY2xvdWQuYXBlcnR1cmVkYXRhLmRldiIsInBuZmVKdnR5cXVwSDdsZ1k4RE5pOVEzZWhUV3kxYTAybXB0Il0="

In [ ]:
from aperturedb.CommonLibrary import create_connector
from aperturedb.Utils import Utils

client = create_connector()
utils = Utils(client)
utils.get_schema()

## Log internal queries

Sometimes to figure out the internal workings of the QA request/response it might be useful to figure out the inner workings. Run the following section to log the internal process.

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import llama_index.core

llama_index.core.set_global_handler("simple")

## Build and query the ApertureDB Property graph

In [ ]:
from typing import Literal
from llama_index.core import PropertyGraphIndex

from llama_index.core import StorageContext
from llama_index.graph_stores.ApertureDB import ApertureDBGraphStore

from llama_index.core.indices.property_graph import SchemaLLMPathExtractor
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

### Define the Schema to guide the LLM

In [ ]:
entities = Literal["Movie", "Genre", "Keyword", "Professional"]
relations = Literal["HasCast", "HasCrew", "HasGenre", "HasKeyword"]

validation_schema = {
    "Person": ["HasCast", "HasCrew", "HasGenre", "HasKeyword"],
    "Movie": ["HasCast", "HasCrew", "HasGenre", "HasKeyword"],
    "Genre": ["HasCast", "HasCrew", "HasGenre", "HasKeyword"],
    "Keyword": ["HasCast", "HasCrew", "HasGenre", "HasKeyword"],
}


### Initialize the Property Graph Index, with an instance of SchemaLLMPathExtractor

In [ ]:
kg_extractor = SchemaLLMPathExtractor(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.0),
    possible_entities=entities,
    possible_relations=relations,
    possible_entity_props=["name", "budget"],
)

graph_store = ApertureDBGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

index = PropertyGraphIndex.from_existing(
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    property_graph_store=graph_store,
    show_progress=True,
)

query_engine = index.as_query_engine()


In [ ]:
def run_queries(query_engine):
    query_str = [
        ("Which movies is Tom Hanks associated with? Give me a list of movies.", True),
        ("Give me a list of movies starring Tom Hanks and Meg Ryan.", True),
        ("What is the genre of the movie 'Forrest Gump'?", True),
        ("Which other actors has Tom Hanks worked with?", True),
        ("What is the budget of the movie 'Forrest Gump'?", False),
        (
            "List all the crew members of the movie 'Forrest Gump', which were not actors.",
            False,
        ),
    ]
    for qs, is_correct in query_str:
        response = query_engine.query(qs)
        print(f"{qs=}")
        print(f"{response.response=}")
        print("=" * 50)


run_queries(query_engine)